<div class="notebook-buttons" style="display:flex; padding-top: 5rem;padding-bottom: 2.5rem;line-height: 2.15;">
    <a href="https://colab.research.google.com/github/zazuko/ssz/blob/main/notebooks/economy.ipynb">
        <div id="colab-link" style="display: flex;padding-right: 3.5rem;padding-bottom: 0.625rem;border-bottom: 1px solid #ececed; align-items: center;">
            <img class="call-to-action-img" src="img/colab.svg" width="30" height="30" style="margin-right: 10px;margin-top: auto;margin-bottom: auto;">
            <div class="call-to-action-txt">Run in Google Colab</div>
        </div>
    </a>
    <a href="https://raw.githubusercontent.com/zazuko/ssz/main/notebooks/economy.ipynb" download>
        <div id="download-link" style="display: flex;padding-right: 3.5rem;padding-bottom: 0.625rem;border-bottom: 1px solid #ececed; height: auto;align-items: center;">
            <img class="call-to-action-img" src="img/download.svg" width="22" height="30" style="margin-right: 10px;margin-top: auto;margin-bottom: auto;">
            <div class="call-to-action-txt">Download Notebook</div>
        </div>
    </a>
    <a href="https://github.com/zazuko/ssz/blob/main/notebooks/economy.ipynb">
        <div id="github-link" style="display: flex;padding-right: 3.5rem;padding-bottom: 0.625rem;border-bottom: 1px solid #ececed; height: auto;align-items: center;">
            <img class="call-to-action-img" src="img/github.svg" width="25" height="30" style="margin-right: 10px;margin-top: auto;margin-bottom: auto;">
            <div class="call-to-action-txt">View on GitHub</div>
        </div>
    </a>
</div>

# Economic activities in Zürich

Zürich Statistical Office collects data on city and its residents. This data is published as [Linked Data](https://en.wikipedia.org/wiki/Linked_data). 

In this tutorial, we will show **how to work with Linked Data.** Mainly, we will see how to work with data on economic activities.   
We will look into how to query, process, and visualize it.   


[1. Restaurants over time](#Restaurants-over-time)  
[2. Restaurants in city quartiers](#Restaurants-in-city-quartiers)  
[3. After-school care: gender-representation](#After-school-care:-gender-representation)

### SPARQL endpoint

Data on some economic activities is published as Linked Data. It can be accessed with [SPARQL queries](https://www.w3.org/TR/rdf-sparql-query/).   
You can send queries using HTTP requests. The API endpoint is **[https://ld.zazuko.com/query/](https://ld.zazuko.com/query/).**  
  
  
Let's use `SparqlClient` from [graphly](https://github.com/zazuko/graphly) to communicate with the database. 
Graphly will allow us to:
* send SPARQL queries
* automatically add prefixes to all queries
* format response to `pandas` or `geopandas`

In [ ]:
# Installing dependencies for Colab environment
!pip install mapclassify
!pip install git+https://github.com/zazuko/graphly.git

In [ ]:
import mapclassify
import matplotlib
import matplotlib.cm

import pandas as pd
import plotly.express as px
import plotly.graph_objects as go

from graphly.api_client import SparqlClient

In [ ]:
ENDPOINT = "https://ld.zazuko.com/query/"

sparql = SparqlClient(ENDPOINT)
sparql.add_prefixes({
    "schema": "<http://schema.org/>",
    "cube": "<https://cube.link/>",
    "property": "<https://ld.stadt-zuerich.ch/statistics/property/>",
    "measure": "<https://ld.stadt-zuerich.ch/statistics/measure/>",
    "skos": "<http://www.w3.org/2004/02/skos/core#>",
    "ssz": "<https://ld.stadt-zuerich.ch/statistics/>"
})

SPARQL queries can become very long. To improve the readibility, we will work wih [prefixes](https://en.wikibooks.org/wiki/SPARQL/Prefixes).
 
Using `add_prefixes` method, we define persistent prefixes. 
Every time you send a query, `graphly` will add automatically update the prefixes for you.

## Restaurants over time

Let's find the number of restaurants in Zurich over time. This information is available in `AST-BTA` data cube. To give restaurants numbers a context, let's scale them by population size. The number of inhabitants over time can be found in `BEW` data cube.

The query for number of inhabitants and restaurants over time is:



In [ ]:
query = """
SELECT *
FROM <https://lindas.admin.ch/stadtzuerich/stat>
WHERE {
    {
    SELECT ?time (SUM(?ast) AS ?restaurants)
    WHERE {
      ssz:AST-BTA a cube:Cube;
                    cube:observationSet/cube:observation ?obs_rest.   
      ?obs_rest property:TIME ?time ;     
           property:RAUM <https://ld.stadt-zuerich.ch/statistics/code/R30000> ;
           property:BTA <https://ld.stadt-zuerich.ch/statistics/code/BTA5000> ;
           measure:AST ?ast . 
    }
     GROUP BY ?time ?place
  }
  {
    SELECT ?time ?pop
    WHERE {
      ssz:BEW a cube:Cube;
                    cube:observationSet/cube:observation ?obs_pop.   
      ?obs_pop property:TIME ?time ;     
           property:RAUM <https://ld.stadt-zuerich.ch/statistics/code/R30000>;
           measure:BEW ?pop
    }
  }  
}
ORDER BY ?time
"""

df = sparql.send_query(query)
df.head()

Let's calculate number of restaurants per 10 000 inhabitants

In [ ]:
df = df.fillna(method="ffill")
df["Restaurants per 10 000 inhabitants"] = df["restaurants"]/df["pop"]*10000

In [ ]:
fig = px.line(df, x="time", y = "Restaurants per 10 000 inhabitants")
fig.update_layout(title_text='Restaurants in Zürich over time', title_x=0.5)

## Restaurants in city quartiers 

Let's find the number of restaurants in different part of the city. The data on restaurants is available in `AST-BTA` data cube. To place the quertiers on the map, we will need their geographic coordinates. This data is available in `Wikidata`. We will get number of restaurants per district from our endpoint, and quertier centroid from `Wikidata`. 

Both information can be obtained using SPARQL [federated query](https://www.w3.org/TR/sparql11-federated-query/). The endpoint for Wikidata is `<https://query.wikidata.org/sparql>`.

The query for quertiers, its centroids, and number of restaurants is:

In [ ]:
# Restaurants per population
query = """
PREFIX p: <http://www.wikidata.org/prop/>
PREFIX ps: <http://www.wikidata.org/prop/statement/>

SELECT ?place ?geometry (SUM(?ast) AS ?restaurants)
WHERE {
  ssz:AST-BTA a cube:Cube;
                  cube:observationSet/cube:observation ?obs.   
  ?obs property:TIME ?time ;     
       property:RAUM ?place_uri ;
       property:BTA <https://ld.stadt-zuerich.ch/statistics/code/BTA5000> ;
                   measure:AST ?ast .
  
  ?place_uri skos:inScheme <https://ld.stadt-zuerich.ch/statistics/scheme/Quartier> ;
             schema:name ?place .

  ?place_uri schema:sameAs ?wikidata_id . 
  
  FILTER (?time = "2017-12-31"^^xsd:date)
  
  BIND(IRI(?wikidata_id ) AS ?wikidata_iri ) .
  
  SERVICE <https://query.wikidata.org/sparql> {
    ?wikidata_iri p:P625/ps:P625 ?geometry .
  }
}
GROUP BY ?place ?geometry ?time
"""

df = sparql.send_query(query)
df.head()

Let's classify the number of restaurants into 5 different buckets. We will use `mapclassify` library to assign values in `restaurant` column into one of five categories.


In [ ]:
N_CATEGORIES = 5
df["text"] = df.place + "<br>Restaurants: " + df.restaurants.astype(int).astype(str)
classifier = mapclassify.NaturalBreaks(y=df["restaurants"], k=N_CATEGORIES)
df["rest_buckets"] = df[["restaurants"]].apply(classifier) 

Classified values can be easily visualized on the map.

In [ ]:
norm = matplotlib.colors.Normalize(vmin=0, vmax=N_CATEGORIES)
colormap = matplotlib.cm.ScalarMappable(norm=norm, cmap=matplotlib.cm.viridis)
labels = mapclassify.classifiers._get_mpl_labels(classifier, fmt="{:.0f}")

fig = go.Figure()

for bucket in range(N_CATEGORIES):

    subset = df[df.rest_buckets == bucket]
    fig.add_trace(go.Scattermapbox(
        mode="markers",
        lat=subset.geometry.y,
        lon=subset.geometry.x,
        hovertext = subset.text,
        hoverinfo = "text",
        name=labels[bucket],
        marker={'size': ((subset.restaurants)**1.5)*0.6, "sizemode": "area", "sizemin": 4, "color": "rgba{}".format(colormap.to_rgba(bucket+1))}, 
    ))

fig.update_layout(
    margin={'l': 0, 't': 50, 'b': 0, 'r': 0},
    mapbox={
        'center': {"lat": 47.3815, "lon": 8.532},
        'style': "carto-darkmatter",
        'zoom': 11},
    showlegend=True,
    legend_title="Restaurants count",
    title_text='Restaurants in Zürich Quartiers', 
    title_x=0.5
)

fig.show()

## After-school care: gender-representation

Let's take a look at gender representation in public sector. In `BES-BTA-SEX` data cube we can find information on number of employees in different organizations. The data is reported separately for each sex, and various establishment types. Let's find the number of male and female employees in after-school care (*Hort*). 

The query for number of female and male employees in after-school care over time is:


In [ ]:
query = """
SELECT ?time ?employees ?sex
FROM <https://lindas.admin.ch/stadtzuerich/stat>
WHERE {
    ssz:BES-BTA-SEX a cube:Cube;
                cube:observationSet/cube:observation ?obs.   
    ?obs property:TIME ?time ;     
        property:RAUM/skos:inScheme <https://ld.stadt-zuerich.ch/statistics/scheme/Gemeinde> ;
        property:BTA/schema:name "Horte" ;
        property:SEX/schema:name ?sex ;
        measure:BES ?employees .
}
ORDER BY ?time
"""
df = sparql.send_query(query)
df.head()

Let's rearrange and rename the columns:

In [ ]:
df = pd.pivot_table(df, index="time", columns="sex", values="employees")
df = df.reset_index().rename_axis(None, axis=1)
df = df.rename(columns={"männlich": "male", "weiblich": "female"})

In [ ]:
fig = px.histogram(df, x="time", y=df.columns, barnorm="percent", labels={'x':'total_bill', 'y':'count'})
fig.update_layout(
    title='After-school care: gender representation', 
    title_x=0.5,
    yaxis_title="% of employees"
)
fig.show()